In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Provideniya,RU,64.38,-173.30,51.80,76,20,11.18
1,1,Husavik,IS,66.04,-17.34,55.40,82,90,6.93
2,2,Vaini,TO,-21.20,-175.20,69.80,88,75,13.87
3,3,Jamestown,US,42.10,-79.24,75.00,60,1,12.75
4,4,Khatanga,RU,71.97,102.50,54.72,77,100,14.74


In [3]:
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [4]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [5]:
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

In [6]:
#1 Assign the locations to an array of latitude and longtiude pairs
locations = city_data_df[["Lat", "Lng"]]

#2 Assign the weights variable to some variables
max_temp = city_data_df["Max Temp"]

#3 Assign the figure variable gmaps.figure() attribute
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

# 4. Assign the heatmap layer to the heatmap attribute and add in the locations
heatmap_layer = gmaps.heatmap_layer(locations, weights=temps, dissipating=False, max_intensity=300, point_radius=4)

#5 Add the heatmap layer
fig.add_layer(heatmap_layer)
#6 Call the figure to plot the data
fig 

Figure(layout=FigureLayout(height='420px'))

In [7]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [8]:
# Heatmap of cloudiness
locations = city_data_df[["Lat", "Lng"]]
clouds = city_data_df["Cloudiness"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=clouds, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [9]:
# Heatmap of Wind Speed
locations = city_data_df[["Lat", "Lng"]]
wind = city_data_df["Wind Speed"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=wind, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [10]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip?"))
max_temp = float(input("What is the maximum temperature you would like for your trip?"))

In [11]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]

preferred_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
13,13,Basudebpur,IN,21.14,86.74,84.85,74,100,11.03
15,15,Arraial Do Cabo,BR,-22.97,-42.03,82.40,51,66,8.05
19,19,Albany,US,42.60,-73.97,90.00,38,8,9.15
29,29,Tucupita,VE,9.06,-62.05,83.95,76,4,4.99
33,33,Kapaa,US,22.08,-159.32,82.40,65,40,16.11


In [12]:
preferred_cities_df.count()

City_ID       110
City          110
Country       110
Lat           110
Lng           110
Max Temp      110
Humidity      110
Cloudiness    110
Wind Speed    110
dtype: int64

In [13]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
13,Basudebpur,IN,84.85,21.14,86.74,
15,Arraial Do Cabo,BR,82.40,-22.97,-42.03,
19,Albany,US,90.00,42.60,-73.97,
29,Tucupita,VE,83.95,9.06,-62.05,
33,Kapaa,US,82.40,22.08,-159.32,
35,Butaritari,KI,83.35,3.07,172.79,
67,Jaciara,BR,84.15,-15.97,-54.97,
75,Haimen,CN,80.60,23.19,116.61,
84,Pandan Niog,PH,81.86,6.23,120.56,
91,Victoria,HK,82.00,22.29,114.16,


In [46]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key}

# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]
# Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_name = hotels["results"][0]["name"]
        #print(hotels["results"][0]["name"])
        hotel_df.loc[index, "Hotel Name"] = hotel_name
    except (KeyError,IndexError):
        print("Hotel not found... Skipping")

Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping


In [37]:
from pprint import pprint
pprint(hotels['results'][0]['name'])

'Holiday Inn Express & Suites Twin Falls'


In [47]:
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
13,Basudebpur,IN,84.85,21.14,86.74,Basudevpur Ice Factory
15,Arraial Do Cabo,BR,82.40,-22.97,-42.03,Pousada Porto Praia
19,Albany,US,90.00,42.60,-73.97,
29,Tucupita,VE,83.95,9.06,-62.05,Hotel Tucupita Su​ite C.A.
33,Kapaa,US,82.40,22.08,-159.32,Sheraton Kauai Resort at Coconut Beach
35,Butaritari,KI,83.35,3.07,172.79,Isles Sunset Lodge
67,Jaciara,BR,84.15,-15.97,-54.97,Taba Hotel
75,Haimen,CN,80.60,23.19,116.61,Jinghao Business Hotel
84,Pandan Niog,PH,81.86,6.23,120.56,
91,Victoria,HK,82.00,22.29,114.16,Mini Hotel Central


In [17]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [18]:
 #Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)

marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(marker_layer)

fig.add_layer(heat_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))